# SystemGenerator replacement

The purpose of this example is to show how a production pipeline reliant on `openmmforcefields` can be migrated to OpenFF infrastructure.

For four different example systems, a cell shows how to prepare it with `openmmforcefields` and then the following cell shows how to prepare an equivalent system with OpenFF infrastructure.`

The imports here are intentionally clunky; since each uses similarly-named classes, only their modules are imported. The result is excessively verbose for most cases, but useful here for clarity.

## Data sources

* [Protein PDB](https://github.com/omsf/joint-demo/blob/8fe4145dd3255c961d04bb16c00f39ecd768dc71/source/openfe/protein.pdb)
* [Ligand(s) SDF](https://github.com/omsf/joint-demo/blob/8fe4145dd3255c961d04bb16c00f39ecd768dc71/source/openfe/ligands.sdf)

## Example #1: Ligand in vacuum

### The "old" way

In [1]:
import openmm.app
import openmm.unit
import openff.toolkit
import openmmforcefields.generators

molecule = openff.toolkit.Molecule.from_smiles(smiles="c1ccccc1")
molecule.generate_conformers(n_conformers=1)

smirnoff_generator = openmmforcefields.generators.SMIRNOFFTemplateGenerator(molecules=molecule, forcefield='openff-2.2.1')
force_field = openmm.app.ForceField()
force_field.registerTemplateGenerator(smirnoff_generator.generator)

force_field.createSystem(
    topology=molecule.to_topology().to_openmm(),
    nonbondedCutoff=9 * openmm.unit.angstrom,
    switchDistance=8 * openmm.unit.angstrom,
    nonbondedMethod=openmm.app.NoCutoff,
)

/Users/mattthompson/micromamba/envs/system-generator-replacement/lib/python3.12/site-packages/openff/utilities/provenance.py:25: CondaExecutableNotFoundWarning: No conda/mamba/micromamba executable found. Unable to determine package versions.
  warnings.warn(
/Users/mattthompson/micromamba/envs/system-generator-replacement/lib/python3.12/site-packages/openff/amber_ff_ports/amber_ff_ports.py:8: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename


<openmm.openmm.System; proxy of <Swig Object of type 'OpenMM::System *' at 0x13a300de0> >

### Using OpenFF infrastructure

In [2]:
import openff.toolkit

molecule = openff.toolkit.Molecule.from_smiles(smiles="c1ccccc1")
molecule.generate_conformers(n_conformers=1)

sage = openff.toolkit.ForceField("openff-2.2.1.offxml")

sage.create_openmm_system(topology=molecule.to_topology())

<openmm.openmm.System; proxy of <Swig Object of type 'OpenMM::System *' at 0x13f8eb630> >

### Example #2: Protein in vacuum

This should$^{\mathrm{TM}}$ be quite simple since there is only one component in the system - just load up a protein-relevant force field and a PDB file, then call `.createSystem()` to squish them together.

As a result, the steps are identical, differing only in how the API points are spelled.

### The "old" way

In [3]:
import openmm.app

pdb = openmm.app.PDBFile("protein.pdb")
ff14sb = openmm.app.ForceField("amber/ff14SB.xml")

ff14sb.createSystem(pdb.topology)

<openmm.openmm.System; proxy of <Swig Object of type 'OpenMM::System *' at 0x152b5bfc0> >

### Using OpenFF infrastructure

In [4]:
import openff.toolkit

topology = openff.toolkit.Topology.from_pdb("protein.pdb")
ff14sb = openff.toolkit.ForceField("ff14sb_off_impropers_0.0.4.offxml")

ff14sb.create_openmm_system(topology)

<openmm.openmm.System; proxy of <Swig Object of type 'OpenMM::System *' at 0x152ccab80> >

### Example #3: Ligand in water

This is much the same as Example #1 - including plenty of re-used code - with the added step of creating solvent around the ligand. There are several ways of doing this; `addSolvent` is one way when things are already represented in memory with OpenMM objects. This requires jumping into `openmm.app.Modeller` which was otherwise unnecessary.

### The "old" way

In [5]:
import openmm.app
import openmm.unit
import openff.toolkit
import openmmforcefields.generators

def make_vec3(positions: openff.toolkit.Quantity) -> openmm.Vec3:
    return [
        openmm.Vec3(float(row[0]), float(row[1]), float(row[2]))
        for row in positions.m_as("nanometer")
    ]
        
molecule = openff.toolkit.Molecule.from_smiles(smiles="c1ccccc1")
molecule.generate_conformers(n_conformers=1)

modeller = openmm.app.Modeller(
    topology =molecule.to_topology().to_openmm(),
    positions=make_vec3(molecule.conformers[0]),
)

smirnoff_generator = openmmforcefields.generators.SMIRNOFFTemplateGenerator(
    molecules=molecule,
    forcefield='openff-2.2.1',
)
force_field = openmm.app.ForceField("tip3p.xml")
force_field.registerTemplateGenerator(smirnoff_generator.generator)

modeller.addSolvent(
    forcefield=force_field,
    boxSize=openmm.Vec3(5, 5, 5),
)

system_generator = openmmforcefields.generators.SystemGenerator()
system_generator.create_system(
    topology=modeller.topology,
    molecules=[molecule, openff.toolkit.Molecule.from_smiles("O")],
)

<openmm.openmm.System; proxy of <Swig Object of type 'OpenMM::System *' at 0x158ebeb50> >

### Using OpenFF infrastructure

In [6]:
import openff.toolkit
from openff.interchange.components._packmol import pack_box


molecule = openff.toolkit.Molecule.from_smiles(smiles="c1ccccc1")
molecule.generate_conformers(n_conformers=1)

water = openff.toolkit.Molecule.from_smiles("O")

packed_topology = pack_box(
    molecules=[molecule, water],
    number_of_copies=[1, 4000],
    target_density=openff.toolkit.Quantity("1 g/cm^3"),
)

sage = openff.toolkit.ForceField("openff-2.2.1.offxml")
sage.create_openmm_system(packed_topology)

<openmm.openmm.System; proxy of <Swig Object of type 'OpenMM::System *' at 0x156634030> >

### Example #4: Protein-ligand complex in solvent

This puts everything together - also adds a bit of salt in with the water.

### The "old" way

In [7]:
import openmm.app
import openff.toolkit

pdb = openmm.app.PDBFile("protein.pdb")
modeller = openmm.app.Modeller(pdb.topology, positions=pdb.positions)

ligand = openff.toolkit.Molecule.from_file("ligands.sdf")[0]
modeller.add(ligand.to_topology().to_openmm(), addPositions=ligand.conformers[0].to_openmm())

smirnoff_generator = openmmforcefields.generators.SMIRNOFFTemplateGenerator(
    molecules=ligand,
    forcefield='openff-2.2.1',
)

force_field = openmm.app.ForceField("amber/ff14SB.xml", "amber/tip3p_standard.xml")
force_field.registerTemplateGenerator(smirnoff_generator.generator)

modeller.addSolvent(
    forcefield=force_field,
    boxSize=openmm.Vec3(5, 5, 5),
)

force_field.createSystem(modeller.topology)

<openmm.openmm.System; proxy of <Swig Object of type 'OpenMM::System *' at 0x1597148a0> >

### Using OpenFF infrastructure

In [8]:
import openff.toolkit
from openff.interchange.components._packmol import solvate_topology, UNIT_CUBE


protein = openff.toolkit.Topology.from_pdb("protein.pdb")
protein.add_molecule(openff.toolkit.Molecule.from_file("ligands.sdf")[0])

# this is no longer just a protein, so rename
protein_ligand_complex = protein

# shift by [3, 3, 3] nm so it ends up in the middle of the box later
protein_ligand_complex.set_positions(
    protein_ligand_complex.get_positions() + openff.toolkit.Quantity([3, 3, 3], "nanometer")
)

topology = solvate_topology(
    topology=protein_ligand_complex,
    nacl_conc=openff.toolkit.Quantity(0.08, 'mole / liter'),
    box_shape=UNIT_CUBE,
    target_density=openff.toolkit.Quantity("1 g/cm^3"),
)

ff14sb_with_sage = openff.toolkit.ForceField("ff14sb_off_impropers_0.0.4.offxml", "openff-2.2.1.offxml")

ff14sb_with_sage.create_openmm_system(topology)

<openmm.openmm.System; proxy of <Swig Object of type 'OpenMM::System *' at 0x15353ab80> >